In [8]:
import pandas as pd
from rdkit import Chem
import seaborn as sns
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

In [5]:
novartis_combined = pd.read_csv("processed/novartis_2024_combined.csv")
novartis_combined.rename(columns={"activity_class": "nov_activity_class"}, inplace=True)

In [14]:
novartis_combined

,SMILES,log_kobs,pIC50_CYP3A4,pIC50_CYP2C9,pIC50_CYP2D6,Source,CANONICAL_SMILES,INCHIKEY,split,data_type,kobs,common_name,dataset,nov_activity_class,action_type
0,CCCCc1nc2cc(/C=C/C(=O)NO)ccc2n1CCN(CC)CC.Cl.Cl,-2.131000,4.957,4.733,4.823,ChEMBL,CCCCc1nc2cc(/C=C/C(O)=NO)ccc2n1CCN(CC)CC,JHDKZFFAIZKUCU-ZRDIBKRKSA-N,train,surrogate,NaN,NaN,novartis_2024,negative,inhibitor
1,CCCCc1ccc(N/C=N/O)cc1,-2.300000,4.597,4.857,4.716,ChEMBL,CCCCc1ccc(NC=NO)cc1,BQPCBRFOSWMALO-UHFFFAOYSA-N,train,surrogate,NaN,NaN,novartis_2024,negative,inhibitor
2,C[C@H](Nc1cc(-c2sc(C3CCN(C)CC3)nc2-c2ccc(F)cc2...,-1.940000,4.740,4.686,4.883,ChEMBL,C[C@H](Nc1cc(-c2sc(C3CCN(C)CC3)nc2-c2ccc(F)cc2...,HJEPFPDXSFKWQH-IBGZPJMESA-N,train,surrogate,NaN,NaN,novartis_2024,weak_positive,inhibitor
3,CC(C)c1nc(-c2ccc(F)cc2)c(-c2ccc3nc(N)n(S(=O)(=...,-2.151000,4.453,4.543,4.525,ChEMBL,CC(C)c1nc(-c2ccc(F)cc2)c(-c2ccc3nc(N)n(S(=O)(=...,OBNKFIQHMAHNEE-UHFFFAOYSA-N,train,surrogate,NaN,NaN,novartis_2024,negative,inhibitor
4,CC(C)S(=O)(=O)n1c(N)nc2ccc(-c3[nH]c(C(C)(C)C)n...,-2.283000,4.372,4.432,4.463,ChEMBL,CC(C)S(=O)(=O)n1c(N)nc2ccc(-c3[nH]c(C(C)(C)C)n...,ABAOSSRHMRVEJF-UHFFFAOYSA-N,train,surrogate,NaN,NaN,novartis_2024,negative,inhibitor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16733,NC2C=CN([C@H]1CC[C@@H](CO)O1)C(=O)N=2,-2.221849,NaN,NaN,NaN,NaN,Nc1ccn([C@H]2CC[C@@H](CO)O2)c(=O)n1,WREGKURFCTUGRC-POYBYMJQSA-N,test,assay,0.006,Zalcitabine,novartis_2024,negative,inhibitor
16734,COc1cc(ccc1OC(F)F)C2C=CC(=O)NN=2,-2.096910,NaN,NaN,NaN,NaN,COc1cc(-c2ccc(=O)[nH]n2)ccc1OC(F)F,HJMQDJPMQIHLPB-UHFFFAOYSA-N,test,assay,0.008,Zardaverine,novartis_2024,negative,inhibitor
16735,CC2=CN([C@H]1C[C@H](N=[N+]=[N-])[C@@H](CO)O1)C...,-2.096910,NaN,NaN,NaN,NaN,Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(...,HBOMLICNUCNMMY-XLPZGREQSA-N,test,assay,0.008,Zidovudine,novartis_2024,negative,inhibitor
16736,CC(C2=Cc1ccccc1S2)N(O)C(N)=O,-2.301030,NaN,NaN,NaN,NaN,CC(c1cc2ccccc2s1)N(O)C(N)=O,MWLSOWXNZPKENC-UHFFFAOYSA-N,test,assay,0.005,Zileuton,novartis_2024,negative,inhibitor


In [6]:
novartis_combined.nov_activity_class.value_counts()

nov_activity_class
negative         8341
weak_positive    5758
positive         2639
Name: count, dtype: int64

In [19]:
classmap = {"negative": 0, "weak_positive":1, "positive":2}

In [21]:
novartis_combined["ACTIVITY_CLASS"] =  novartis_combined["nov_activity_class"].apply(lambda x: classmap[x])

In [23]:
novartis_combined


,SMILES,log_kobs,pIC50_CYP3A4,pIC50_CYP2C9,pIC50_CYP2D6,Source,CANONICAL_SMILES,INCHIKEY,split,data_type,kobs,common_name,dataset,nov_activity_class,action_type,ACTIVITY_CLASS
0,CCCCc1nc2cc(/C=C/C(=O)NO)ccc2n1CCN(CC)CC.Cl.Cl,-2.131000,4.957,4.733,4.823,ChEMBL,CCCCc1nc2cc(/C=C/C(O)=NO)ccc2n1CCN(CC)CC,JHDKZFFAIZKUCU-ZRDIBKRKSA-N,train,surrogate,NaN,NaN,novartis_2024,negative,inhibitor,0
1,CCCCc1ccc(N/C=N/O)cc1,-2.300000,4.597,4.857,4.716,ChEMBL,CCCCc1ccc(NC=NO)cc1,BQPCBRFOSWMALO-UHFFFAOYSA-N,train,surrogate,NaN,NaN,novartis_2024,negative,inhibitor,0
2,C[C@H](Nc1cc(-c2sc(C3CCN(C)CC3)nc2-c2ccc(F)cc2...,-1.940000,4.740,4.686,4.883,ChEMBL,C[C@H](Nc1cc(-c2sc(C3CCN(C)CC3)nc2-c2ccc(F)cc2...,HJEPFPDXSFKWQH-IBGZPJMESA-N,train,surrogate,NaN,NaN,novartis_2024,weak_positive,inhibitor,1
3,CC(C)c1nc(-c2ccc(F)cc2)c(-c2ccc3nc(N)n(S(=O)(=...,-2.151000,4.453,4.543,4.525,ChEMBL,CC(C)c1nc(-c2ccc(F)cc2)c(-c2ccc3nc(N)n(S(=O)(=...,OBNKFIQHMAHNEE-UHFFFAOYSA-N,train,surrogate,NaN,NaN,novartis_2024,negative,inhibitor,0
4,CC(C)S(=O)(=O)n1c(N)nc2ccc(-c3[nH]c(C(C)(C)C)n...,-2.283000,4.372,4.432,4.463,ChEMBL,CC(C)S(=O)(=O)n1c(N)nc2ccc(-c3[nH]c(C(C)(C)C)n...,ABAOSSRHMRVEJF-UHFFFAOYSA-N,train,surrogate,NaN,NaN,novartis_2024,negative,inhibitor,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16733,NC2C=CN([C@H]1CC[C@@H](CO)O1)C(=O)N=2,-2.221849,NaN,NaN,NaN,NaN,Nc1ccn([C@H]2CC[C@@H](CO)O2)c(=O)n1,WREGKURFCTUGRC-POYBYMJQSA-N,test,assay,0.006,Zalcitabine,novartis_2024,negative,inhibitor,0
16734,COc1cc(ccc1OC(F)F)C2C=CC(=O)NN=2,-2.096910,NaN,NaN,NaN,NaN,COc1cc(-c2ccc(=O)[nH]n2)ccc1OC(F)F,HJMQDJPMQIHLPB-UHFFFAOYSA-N,test,assay,0.008,Zardaverine,novartis_2024,negative,inhibitor,0
16735,CC2=CN([C@H]1C[C@H](N=[N+]=[N-])[C@@H](CO)O1)C...,-2.096910,NaN,NaN,NaN,NaN,Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(...,HBOMLICNUCNMMY-XLPZGREQSA-N,test,assay,0.008,Zidovudine,novartis_2024,negative,inhibitor,0
16736,CC(C2=Cc1ccccc1S2)N(O)C(N)=O,-2.301030,NaN,NaN,NaN,NaN,CC(c1cc2ccccc2s1)N(O)C(N)=O,MWLSOWXNZPKENC-UHFFFAOYSA-N,test,assay,0.005,Zileuton,novartis_2024,negative,inhibitor,0


In [24]:

ncats_adme = pd.read_csv("./processed/ncats_adme_aid1645841.csv")

In [25]:
ncats_adme

,CANONICAL_SMILES,INCHIKEY,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_CID,dataset,data_type,common_name,action_type
0,Cc1ccc2c(c1)c1c(n2CCc2ccc(C)nc2)CCN(C)C1,JNODQFNWMXFMEV-UHFFFAOYSA-N,Active,197033,ncats_adme_aid1645841,assay,NaN,inhibitor
1,O=c1c(O)c(-c2ccc(O)cc2O)oc2cc(O)cc(O)c12,YXOLAZRVSSWPPT-UHFFFAOYSA-N,Inactive,5281670,ncats_adme_aid1645841,assay,NaN,inhibitor
2,Cc1ccc(C(=O)NC2C(=O)NC(C(C)C)C(=O)N3CCCC3C(=O)...,GQBHKSFHNFMVES-TYWDLNKHSA-N,Inconclusive,6604832,ncats_adme_aid1645841,assay,NaN,inhibitor
3,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,KZNIFHPLKGYRTM-UHFFFAOYSA-N,Active,5280443,ncats_adme_aid1645841,assay,NaN,inhibitor
4,NC(Cc1ccc(O)c(O)c1)C(=O)O,WTDRDQBEARUVNC-UHFFFAOYSA-N,Active,836,ncats_adme_aid1645841,assay,NaN,inhibitor
...,...,...,...,...,...,...,...,...
5233,C=CCN1CCC23c4c5ccc(O)c4OC2C(=O)CC[C@@]3(O)C1C5,UZHSEJADLWPNLE-AQBRWLJTSA-N,Inactive,11957637,ncats_adme_aid1645841,assay,NaN,inhibitor
5234,CCCNC(C)C(=O)Nc1ccccc1C,MVFGUOIZUNYYSO-UHFFFAOYSA-N,Inactive,4906,ncats_adme_aid1645841,assay,NaN,inhibitor
5235,CCN(Cc1ccncc1)C(=O)C(CO)c1ccccc1,BGDKAVGWHJFAGW-UHFFFAOYSA-N,Active,5593,ncats_adme_aid1645841,assay,NaN,inhibitor
5236,C[N+](C)(C)CCCCCCCCCC[N+](C)(C)C,MTCUAOILFDZKCO-UHFFFAOYSA-N,Inconclusive,2968,ncats_adme_aid1645841,assay,NaN,inhibitor


In [ ]:
classmap = {"Inactive": 0, "Inconclusive": 1, ""}